# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [3]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [5]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [11]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [12]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/THUDM/GLM-4.1V-9B-Thinking',
 '/kyutai/tts-1.6b-en_fr',
 '/google/gemma-3n-E4B-it',
 '/apple/DiffuCoder-7B-cpGRPO',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/AIDC-AI/Ovis-U1-3B',
 '/spaces/kontext-community/FLUX.1-Kontext-portrait',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/facebook/seamless-interaction',
 '/datasets/marcelbinz/Psych-101',
 '/datasets/black-forest-labs/kontext-bench',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Wri

In [18]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [19]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [21]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Claude
Chat with Claude
Overview
Max plan
Team plan
Enterprise plan
Education plan
Explore pricing
Download apps
Claude log in
News
Claude’s character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Learn how to build with Claude
Solutions
Collaborate with Claude
AI agents
Coding
Customer support
Case studies
Hear from our customers
Research
Research
Overview
Economic Index
Claude model family
Claude Opus 4
Claude Sonnet 4
Claude Haiku 3.5
Research
Claude’s extended thinking
Commitments
Initiatives
Transparency
Responsible scaling policy
Trust center
Security and compliance
Announcement
ISO 42001 certification
Learn
Learning resources
Customer stories
Engineering at Anthropic
Anthropic Acad

In [35]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [36]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [37]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'learning page', 'url': 'https://www.anthropic.com/learn'}]}


'You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nChat with Claude\nOverview\nMax plan\nTeam plan\nEnterprise plan\nEducation plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCoding\nCustomer support\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\nClaude model family\nClaude Opus 4\nClaude Sonnet 4\nClaude Haiku 3.5\nResearch\nClaude’s extended thinking\nCommitments\nInitiatives\nTransparency\nResponsible scaling policy\nTrust center\nSecurity and compliance\nAnnouncement\nISO\xa042001 certi

In [38]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [39]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'home page', 'url': 'https://www.anthropic.com/'}, {'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}]}


# Welcome to Anthropic: The AI Playground!

## 🌟 Meet Claude: Your New Digital Buddy!
Introducing Claude Opus 4! The latest in AI models that doubles as your chat partner, coding guru, and occasional existential crisis consultant—because who doesn’t need a little AI wisdom while pondering the universe? 🤖✨

## 🏢 Company Culture: A Playlist of Innovation
At Anthropic, we don’t just work hard; we work smart! Picture a vibrant space filled with innovative thinkers sipping on ethically-sourced coffee while engaging in serious discussions about AI safety (and maybe some lighthearted debates about who would win in a race: a cheetah or Claude during a coding sprint). 

**Core Values:**
- **Safety First!**: We believe in justly handling AI’s superhero powers while keeping ethics on our short list of priorities (right after coffee).
- **Learning Together**: With the Anthropic Academy, we empower our crew to teach and learn from our AI buddy, Claude—who's only slightly more intelligent than your average goldfish!

## 👥 Customers: The Humans of the Future
Anthropic serves a diverse clientele who see the potential in AI-powered applications, spanning education to enterprise solutions. Whether you’re a student wanting to ace that tough assignment or a business trying to decode market trends, Claude's got your back! Plus, our customers report increased productivity—no robots were harmed in the process!

## 💼 Careers at Anthropic: Join Our Quest!
Ready to unleash your inner AI wizard? Check out our job openings! We’re hunting for talented individuals who aren't afraid to dream big about the future of AI. Ideal candidates are those who know the difference between a neural network and a band—with a passion for building a safer, smarter world (and a decent sense of humor).

### Current Open Positions:
- AI Safety Evangelist (must have a cape)
- Data Scientist (loves digging through numbers like searching for treasure)
- Software Engineer (JavaScript proficient, risk-taker, and ice cream lover)

## 🤔 Why Choose Anthropic?
- **Responsible AI**: We take our responsibility to society seriously (cue the superhero cape).
- **Tech for Good**: We focus on the long-term well-being of humanity—because why not save the world while sipping coffee?
- **Epic Collaborations**: Work together with Claude (no, he’s not a Snooze Button—Claude will actually help you).

---

**Join Anthropic today and help us shape a safer AI-driven future while having a barrel of laughs!** 

*P.S. No actual barrels were harmed in the making of this brochure.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [40]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [41]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'about page', 'url': 'https://www.anthropic.com/about'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}]}


# Welcome to Anthropic! 🌍🤖

## Where AI Meets Responsibility and a Dash of Wit!

### Who Are We? 🤔
Welcome to Anthropic, the realm of Claude—your friendly neighborhood AI that promises to serve humanity without plotting world domination... at least for now! We emphasize the importance of building powerful technologies with human benefit in mind (and a sprinkle of humor). 

### Meet Claude! 🎩
Say hello to **Claude Opus 4** and its witty siblings, **Claude Sonnet 4** and **Claude Haiku 3.5**! ⚡ These aren’t just names for the hottest new bands—these are our AI models pushing the frontiers of coding, collaboration, and AI agents! Who knew a "sonnet" could also know your coffee order? 

### Our Ethos: Responsible AI 🛡️
**AI Safety is Our Middle Name** (well, not really, but it should be)! Our team takes cutting-edge AI seriously, which is why we pause to ensure it serves humanity's long-term well-being (right after a quick joke). We believe in responsible scaling because trust is great, but it doesn’t grow on trees!

### Join the Fun! 🎉
Looking to team up with geniuses who aren’t afraid to laugh while changing the world? Anthropic has open roles where you can collaborate with Claude and help us build the future of **safe AI**! 

*And remember, if the robots ever take over, it’s only because you've trained them too well!*

### Our Customers: Who We Serve 🏆
Whether you're a curious entrepreneur or an education institution, our customers adore collaborating with Claude for coding, customer support, and everything in-between! Don't believe us? Just check our extensive list of **Customer Stories**. They might even have more fans than Claude!

### Perks of Joining Anthropic 🥳
- **Flexible Plans**: Whether you’re a lone wolf with our Max Plan or a powerhouse with our Enterprise Plan, we’ve got you covered! 
- **Anthropic Academy**: Learn how to build with Claude—because who said AI development couldn’t be educational? 
- **Innovation Central**: Work alongside brilliant minds in a culture that values humor, progress, and the occasional coffee break.

---

### Ready to Have Fun and Work Safely with AI? 
Jump on board with **Anthropic** today and let’s build something magnificent together—after all, Claude’s thinking might just inspire you to press that “try Claude” button!  

---

*Disclaimer: Any robots developed during your time with Anthropic should not be held responsible for any puns or dad jokes that may ensue. Proceed at your humor risk!* 😄

In [42]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'about page', 'url': 'https://www.anthropic.com/about'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'education page', 'url': 'https://www.anthropic.com/education'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}]}



# Welcome to Anthropic: AI with a Smile! 🤖✨

**Who We Are:**
At Anthropic, our guiding principle is simple: We believe in AI that cares! (Like your pet dog, but way smarter.) With our flagship AI, Claude, we prioritize safety, responsibility, and innovation. Forget about evil robots taking over the world—Claude's here to help, not to conquer!

---

## Meet Claude: Your Friendly AI Sidekick 🦸‍♂️

Introducing **Claude Opus 4**, the genius behind the screen! 🤓 Whether you need coding help, customer support, or just a good dad joke, Claude has got your back. Chat with Claude to dive deep into AI magic or just to argue about which pizza topping reigns supreme. (We all know it's pineapple!)

### Claude's Capabilities:
- **Coding Wizardry:** Need to whip up some code? Claude can conjure up programs faster than you can say “syntax error.”
- **Problem Solver:** Have a burning question? Ask Claude. Just please don’t ask for relationship advice—those algorithms can only go so far!

---

## Company Culture: Hugs and Higgs Bosons 🌌❤️

At Anthropic, we strive to blend the brilliance of AI with the warmth of human touch. Our culture is built on:
- **Collaboration:** Opinions and ideas are welcome (including the wild ones—bring on the conspiracy theories!).
- **Transparency:** We promise to keep you in the loop—like that friend who always gives you the gossip (but with less drama).
- **Commitment to Safety:** We’re dead serious about being careful in our AI development. No rogue robots here—just a team that wants to see humanity thrive.

---

## Becoming Part of the Anthropic Family 👩‍💻👨‍💻

### Careers at Anthropic:
Are you ready to join a team that puts the "fun" in "functional AI"? (Okay, maybe that’s a stretch but you get the idea.) We’re looking for:
- **AI Researchers:** Have a PhD in sorts? Great! We’ll supply the coffee.
- **Engineers:** Bring us your best coding chops, and we’ll show you our snack stash.
- **Customer Support Superstars:** Love helping people? Come be Claude’s best buddy.

You’ll get the chance to build the future of safe AI while enjoying a culture that genuinely cares about your well-being. Yes, you can wear your slippers to work (on Zoom days).

---

## Don’t Just Take Our Word for It! 🎤

**Hear from our Customers:**
“Claude helped me complete three projects in half the time! It’s like having a supercharged intern, but they never need a coffee break!” – Happy Customer

---

### Join Us on this Intelligent Adventure! 🌟

Curious to learn more? Schedule a chat with our team or visit our careers page. Together, we can create AI that makes the world a better place—along with the best cheese jokes you'll ever hear!

**Let’s build a brighter future… with Claude by our side!**



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>